## Analys av Bolagsverkets statistik över försenade årsredovisningar 2004-2018

av [Anna Norberg](www.twitter.com/annanorberg)  

Grunddata är Bolagsverkets öppna data. Resultatet från analysen har använts som underlag till en artikel som skickades ut till Nyhetsbyrån Sirens kunder den 26 september 2019.  

In [38]:
library(writexl)
library(dplyr)
library(here)
library(readr)
library(openxlsx)
library(reshape2)

**Läs in data.**

In [39]:
raw <- read_delim("https://www.bolagsverket.se/statistik/rakenskaps_forsening.csv", ";", locale=locale(encoding="ISO-8859-1")) 
df <- raw %>% 
  select(Period_tom_DT,
         Lan_NM,
         Ska_skicka_in_CNT,
         Forseningsavgift_CNT,
         Antal_inkomna_arsred_CNT,
         Andel_forsningsavgift_CNT) %>% 
  rename(År=Period_tom_DT, Län=Lan_NM) %>%
    filter(År < 2019 & År > 2003)

Parsed with column specification:
cols(
  Period_tom_DT = col_integer(),
  Rakenskapsperiod_grupperad_CD = col_character(),
  Lan_NM = col_character(),
  Ska_skicka_in_CNT = col_integer(),
  Forseningsavgift_CNT = col_integer(),
  Antal_inkomna_arsred_CNT = col_integer(),
  Andel_arsredovisning_CNT = col_character(),
  Andel_forsningsavgift_CNT = col_character()
)


In [40]:
head(raw)

Period_tom_DT,Rakenskapsperiod_grupperad_CD,Lan_NM,Ska_skicka_in_CNT,Forseningsavgift_CNT,Antal_inkomna_arsred_CNT,Andel_arsredovisning_CNT,Andel_forsningsavgift_CNT
2003,12-31,Stockholms län,45661,3027,46726,"1,0233","0,0663"
2003,12-31,Uppsala län,3340,128,3410,"1,0210","0,0383"
2003,12-31,Södermanlands län,3182,116,3226,"1,0138","0,0365"
2003,12-31,Östergötlands län,4731,126,4840,"1,0230","0,0266"
2003,12-31,Jönköpings län,5072,120,5174,"1,0201","0,0237"
2003,12-31,Kronobergs län,2310,57,2357,"1,0203","0,0247"


**Årssammanställning.**

Hur har utvecklingenAndelen förseningsavgifter beräknas genom att dividera antalet förseningsavgifter med antalet årsredovisningar som ska skickas in.

In [41]:
year <- df %>%
  group_by(År) %>% 
  summarize("Antal som ska skicka in"=sum(Ska_skicka_in_CNT, na.rm=TRUE),
            "Inkomna årsredovisningar"=sum(Antal_inkomna_arsred_CNT, na.rm=TRUE),
            "Förseningsavgifter"=sum(Forseningsavgift_CNT, na.rm=TRUE),
            "Andel förseningsavgifter"=sprintf("%.1f%%", (sum(Forseningsavgift_CNT, na.rm=TRUE)/sum(Ska_skicka_in_CNT, na.rm=TRUE))*100))

In [42]:
head(year, 20)

År,Antal som ska skicka in,Inkomna årsredovisningar,Förseningsavgifter,Andel förseningsavgifter
2004,266689,276028,9543,3.6%
2005,273540,282829,10648,3.9%
2006,283813,293179,11339,4.0%
2007,296188,304248,12387,4.2%
2008,309871,313865,12784,4.1%
2009,319544,323384,12827,4.0%
2010,333527,336847,14305,4.3%
2011,361701,364495,15749,4.4%
2012,389232,392355,15812,4.1%
2013,411031,414284,15868,3.9%


**I vilket län är det vanligast med försenade årsredovisningar? Länssammanställning för 2018.**

In [43]:
lan <- df %>% filter(År==2018) %>% 
  group_by(Län) %>% 
  summarize("Antal som ska skicka in"=sum(Ska_skicka_in_CNT, na.rm=TRUE),
            "Inkomna årsredovisningar"=sum(Antal_inkomna_arsred_CNT, na.rm=TRUE),
            "Förseningsavgifter"=sum(Forseningsavgift_CNT, na.rm=TRUE),
            "Andel förseningsavgifter"=sprintf("%.1f%%", (sum(Forseningsavgift_CNT, na.rm=TRUE)/sum(Ska_skicka_in_CNT, na.rm=TRUE))*100)) %>% 
  arrange(desc(`Andel förseningsavgifter`))

In [44]:
head(lan, 25)

Län,Antal som ska skicka in,Inkomna årsredovisningar,Förseningsavgifter,Andel förseningsavgifter
Stockholms län,187075,185687,8546,4.6%
Uppsala län,16390,16280,674,4.1%
Skåne län,65536,65130,2406,3.7%
Södermanlands län,11650,11549,411,3.5%
Västmanlands län,10810,10739,381,3.5%
Norrbottens län,10214,10150,344,3.4%
Örebro län,11741,11636,398,3.4%
Gävleborgs län,11117,11078,351,3.2%
Gotlands län,3131,3107,97,3.1%
Kronobergs län,8229,8212,259,3.1%


**Förseningsavgifterna ökar för varje år i riket, men hur ser det ut på länsnivå?**

In [45]:
lan_peryear <- df %>% select(År, Län, Forseningsavgift_CNT) %>%
  dcast(Län ~ År, fun.aggregate=sum)

Using Forseningsavgift_CNT as value column: use value.var to override.


In [46]:
head(lan_peryear, 25)

Län,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
Blekinge län,53,67,81,71,68,66,84,124,94,106,110,120,130,133,141
Dalarnas län,141,159,192,166,184,177,208,237,295,276,293,294,279,291,316
Gotlands län,39,48,63,61,69,54,65,92,83,70,69,70,94,88,97
Gävleborgs län,112,143,168,180,150,168,218,222,243,250,303,308,291,319,351
Hallands län,158,174,184,230,212,249,267,321,323,329,299,341,359,434,412
Jämtlands län,84,110,109,126,138,158,146,180,186,190,155,157,170,188,212
Jönköpings län,159,169,199,217,210,218,236,267,288,275,274,300,347,420,399
Kalmar län,101,125,117,158,181,180,213,219,188,192,220,218,190,252,261
Kronobergs län,81,87,87,107,112,98,113,115,152,142,135,173,185,254,259
Norrbottens län,177,176,157,144,188,239,243,278,290,308,315,343,340,313,344


**Gör en tabell för varje län.**

In [47]:
lista_lan <- c("Blekinge län", "Dalarnas län", "Gotlands län", "Gävleborgs län", "Hallands län", "Jämtlands län", "Jönköpings län", "Kalmar län", "Kronobergs län", "Norrbottens län", "Skåne län", "Stockholms län", "Södermanlands län", "Uppsala län", "Värmlands län", "Västerbottens län", "Västernorrlands län", "Västmanlands län", "Västra Götalands län", "Örebro län", "Östergötlands län")

for (i in lista_lan) {
  table <- df %>% filter(Län == i) %>% 
  group_by(År) %>%
  summarize("Ska skicka in"=sum(Ska_skicka_in_CNT, na.rm=TRUE),
            "Inkomna årsredovisningar"=sum(Antal_inkomna_arsred_CNT, na.rm=TRUE),
            "Förseningsavgifter"=sum(Forseningsavgift_CNT, na.rm=TRUE),
            "Andel förseningsavgifter"=sprintf("%.1f%%", (sum(Forseningsavgift_CNT, na.rm=TRUE)/sum(Ska_skicka_in_CNT, na.rm=TRUE))*100))
  assign(i, table)
}

**Ett par exempel på länstabellerna.**

In [48]:
head(`Blekinge län`, 20)

År,Ska skicka in,Inkomna årsredovisningar,Förseningsavgifter,Andel förseningsavgifter
2004,2957,3079,53,1.8%
2005,3005,3139,67,2.2%
2006,3088,3204,81,2.6%
2007,3202,3291,71,2.2%
2008,3266,3333,68,2.1%
2009,3346,3403,66,2.0%
2010,3506,3537,84,2.4%
2011,3754,3760,124,3.3%
2012,4001,4032,94,2.3%
2013,4217,4236,106,2.5%


In [49]:
head(`Dalarnas län`, 20)

År,Ska skicka in,Inkomna årsredovisningar,Förseningsavgifter,Andel förseningsavgifter
2004,6896,7109,141,2.0%
2005,7058,7289,159,2.3%
2006,7179,7455,192,2.7%
2007,7393,7595,166,2.2%
2008,7661,7778,184,2.4%
2009,7896,8032,177,2.2%
2010,8278,8386,208,2.5%
2011,8960,9034,237,2.6%
2012,9541,9639,295,3.1%
2013,9876,9978,276,2.8%


In [50]:
head(`Stockholms län`, 20)

År,Ska skicka in,Inkomna årsredovisningar,Förseningsavgifter,Andel förseningsavgifter
2004,84349,87715,4811,5.7%
2005,86513,89805,5214,6.0%
2006,89754,93283,5484,6.1%
2007,94119,97200,6005,6.4%
2008,98719,100093,6019,6.1%
2009,101959,103354,6169,6.1%
2010,106807,107984,6746,6.3%
2011,116976,117863,7339,6.3%
2012,126906,127936,7213,5.7%
2013,134693,135875,7242,5.4%


In [51]:
head(`Skåne län`, 20)

År,Ska skicka in,Inkomna årsredovisningar,Förseningsavgifter,Andel förseningsavgifter
2004,32596,33729,937,2.9%
2005,33566,34674,1165,3.5%
2006,35005,36140,1292,3.7%
2007,36658,37654,1438,3.9%
2008,38444,38928,1591,4.1%
2009,39655,40084,1492,3.8%
2010,41121,41532,1735,4.2%
2011,44477,44763,1828,4.1%
2012,47611,47897,1856,3.9%
2013,50279,50566,1836,3.7%


In [52]:
head(`Hallands län`, 20)

År,Ska skicka in,Inkomna årsredovisningar,Förseningsavgifter,Andel förseningsavgifter
2004,7853,8065,158,2.0%
2005,8099,8336,174,2.1%
2006,8489,8705,184,2.2%
2007,8897,9121,230,2.6%
2008,9483,9612,212,2.2%
2009,9885,9977,249,2.5%
2010,10465,10541,267,2.6%
2011,11425,11500,321,2.8%
2012,12411,12496,323,2.6%
2013,13131,13236,329,2.5%
